Action performed:
- PDFminer for pdf to txt conversion, for all input pdfs
- From txt, description using nltk
- summarization using nltk

In [1]:
# general imports
from pathlib import Path
import os

# processing imports
import pandas as pd

In [2]:
# utils elements to move to utils after development

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import HTMLConverter,TextConverter,XMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import io

def convert(fname, pages=None):
    if not pages: pagenums = set();
    else:         pagenums = set(pages);      
    manager = PDFResourceManager() 
    codec = 'utf-8'
    caching = True

    output = io.StringIO()
    converter = TextConverter(manager, output, codec=codec, laparams=LAParams())     

    interpreter = PDFPageInterpreter(manager, converter)   
    infile = open(fname, 'rb')

    for page in PDFPage.get_pages(infile, pagenums,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    convertedPDF = output.getvalue()  

    infile.close(); converter.close(); output.close()
    return convertedPDF

def getListOfFiles(dirName):
    '''
        For the given path, get the List of all files in the directory tree 
    '''
    paths = []
    for path, subdirs, files in os.walk(dirName):
        for name in files:
            paths.append((Path(path+name)))            
    return paths

In [3]:
# start with one pdf
input_path = "../../data/input/DPEFs/"
output_path = "../../data/processed/DPEFs/"
all_input_files = getListOfFiles(input_path)
all_input_files = [p for p in all_input_files if p.name.lower().endswith(".pdf")]
# input_filename = "Construction/vinci-rapport-annuel-2018.pdf"
# output_filename = input_filename.replace(".pdf",".txt")
# input_filename = Path(input_path+input_filename)
# output_filename = Path(output_path+output_filename)

In [29]:
# TODO: replace "\n\n" by " "
# TODO: parse pages paragraphs by paragraphs and not line by line
# accrding to answer in https://stackoverflow.com/questions/53219016/detecting-sections-of-a-pdf-with-pdfminer
# it is not possibmle to parse content in a randomly structured pdf.
for i, input_file in enumerate(all_input_files):
    if input_file.name.endswith("pdf"):
        print("Processing {}/{} - file: {}".format(i,len(all_input_files),input_file.name))
        pdf_full_txt = convert(input_file)
        # create the output folder if needed
        output_file = Path(str(input_file).replace(".pdf",".txt").replace("input","processed"))
        output_file.parents[0].mkdir(parents=True, exist_ok=True)
        fileConverted = open(output_file, "w", encoding="utf-8")
        fileConverted.write(pdf_full_txt)
        fileConverted.close()
        if i == 0:
            break # remove to do all of them

Processing 0/15 - file: LVMH - rapport-rse-2018.pdf


In [25]:
print(pdf_full_txt[10000:12000].replace("\n\n","|"))

plus noble et de plus abouti, une attention |lutte contre la corruption.|minutieuse est portée aux détails et à la perfection : du produit |LVMH soutient également la Déclaration universelle des droits de |au service, c’est dans cette quête d’excellence que le Groupe |l’homme, les Principes directeurs de l’Organisation de coopéra-|cultive sa différence.|tion et de développement économiques (OCDE), les conventions |fondamentales de l’Organisation internationale du travail, les |Cultiver l’esprit d’entreprise
Agile, l’organisation décentralisée du Groupe favorise l’efficacité |principes d’autonomisation des femmes des Nations unies ainsi |que la Charte de la diversité. La stratégie de responsabilité sociale |et la réactivité. Elle stimule les initiatives individuelles en confiant |du Groupe contribue également à l’atteinte des 17 objectifs de |des responsabilités importantes à chacun. L’esprit d’entreprise |développement durable (ODD) des Nations unies.|promu par le Groupe facilite la pr

In [12]:
from summa.summarizer import summarize
summarize(pdf_full_txt, words=200, split=True, language="french")

['et de la lutte contre les discriminations, ',
 'LVMH et ont, au fil du temps, assuré le succès des Maisons et ',
 'Le Code de Conduite LVMH, adopté en 2009 et révisé en 2017, ',
 'évolutions de la société, analyse les risques et les enjeux liés à ',
 'DANS LA CULTURE DU GROUPE LVMH, À TOUTES LES ÉTAPES DU PARCOURS DES COLLABORATEURS.',
 'environnement de travail épanouissant pour les femmes et à ',
 'LA GESTION ET LE DÉVELOPPEMENT DE SES COLLABORATEURS SENIORS.',
 'carrière a été proposé à tous les seniors âgés de 55 ans et plus, ',
 'maintenir les seniors en emploi et d’assurer la transmission des ',
 'LA MISSION HANDICAP, LES ENGAGEMENTS DU GROUPE ET DE SES MAISONS  ',
 'À l’initiative de LVMH et du pôle événement des Echos, le Salon ',
 'matériel d’écriture et le distribuer à des élèves en situation de ',
 'DES SAVOIR-FAIRE ET DANS LE SOUTIEN À LA CRÉATION, AVEC L’OBJECTIF  ',
 '–  La FacStory et le Creative Lab de la Maison Guerlain, qui  ',
 'la Maison a également formé son Comi

In [ ]:
keywords(pdf_full_txt,words=50, split="\n", scores=False, pos_filter=('NN', 'JJ'), lemmatize=True, deacc=True)
# .split('\n')